In [3]:
!pip install pandas

In [4]:
# imports
import pandas as pd

# **Assignment - Data Cleaning**

Group: #45

Isaac Lafond - 300191954

## **Dataset 1 - Clean Data checking**

In [5]:
# Read dataset from public github repo
df1 = pd.read_csv("https://raw.githubusercontent.com/IsaacLafond/CSI-4142---Fundamentals-of-Data-Science/main/Assignment%202/datasets/dirty_cafe_sales.csv")

### **1) Data Type errors**

Data type check:

In this test we verify that the data entered in the columns follow a coherent datatype and count the occurrences of incorrect datatypes.

In [6]:
# Parameters
columns1 = ["Item", "Quantity", "Price Per Unit", "Total Spent", "Payment Method", "Location", "Transaction Date"]
# Notice transaction ID is not include as all the values are present and unique

test_column1 = "Item"

In [7]:
# Checker code
df1.info()
# List of invalid value types
invalid = ["ERROR", "UNKNOWN"]

def num_valid_categories(col, invalid):
  # remove values from given column that aren't invalid, not na and not null and returning the count the of column
  return df1[~df1[col].isin(invalid) & df1[col].notna() & df1[col].notnull()][col].count() #[3]

def num_valid_numeric(col):
  # Convert column to numeric and force error to be NAN which aren't counted in final column and return count
  return pd.to_numeric(df1[col], errors="coerce").count() #[1]

def num_valid_date(col):
  return pd.to_datetime(df1[col], errors="coerce").count() #[2]

# Dictionnary of predicate the count error values
valid_values = {
    "Item": num_valid_categories("Item", invalid),
    "Quantity": num_valid_numeric("Quantity"),
    "Price Per Unit": num_valid_numeric("Price Per Unit"),
    "Total Spent": num_valid_numeric("Total Spent"),
    "Payment Method": num_valid_categories("Payment Method", invalid),
    "Location": num_valid_categories("Location", invalid),
    "Transaction Date": num_valid_date("Transaction Date")
}

valid_values[test_column1]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Transaction ID    10000 non-null  object
 1   Item              9667 non-null   object
 2   Quantity          9862 non-null   object
 3   Price Per Unit    9821 non-null   object
 4   Total Spent       9827 non-null   object
 5   Payment Method    7421 non-null   object
 6   Location          6735 non-null   object
 7   Transaction Date  9841 non-null   object
dtypes: object(8)
memory usage: 625.1+ KB


9031

**Results:**

The following is the the final counts for each column after having removed invalid values from categorical and date data and remove invalid and NAN/null values from numerical data.

Item: 9031

Quantity: 9521

Price Per Unit: 9467

Total Spent: 9498

Payment Method: 6822

Location: 6039

Transaction Date: 9540

### **2) Range errors**

Range check:

In this test, we will verify the range of a numerical value. The range is the minimum and maximum values that an attribute can have.

In [26]:
columns2 = ["Quantity", "Price Per Unit", "Total Spent", "Payment Method"]
# Attribute selection
test_attribute = "Quantity"
# Minimum (e.g. 0$)
minimum = 0
# Maximum (e.g. 200K$)
maximum = 10

In [28]:
df1_num = df1[[test_attribute]].copy()  # Keep it as a DataFrame
df1_num[test_attribute] = pd.to_numeric(df1_num[test_attribute], errors="coerce")

df_filtered = df1_num[~(df1_num[test_attribute] > minimum) & (df1_num[test_attribute] < maximum)]
df_filtered

,Quantity


### **3) Format errors**

### **4) Consistency errors**

### **5) Uniqueness errors**

### **6) Presence errors**

### **7) Length errors**

### **8) Look-up errors**

### **9) Exact duplicate errors**

### **10) Near duplicate errors**

# **References**

[1] https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_numeric.html

[2] https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.to_datetime.html

[3] ChatGPT: Prompt = "Pandas allows for removing values from a dataset based on condition like the following: df[df["age"] > 0] to remove age values 0 or less. Can you write a similar condition which removes values that are null or equal to the values in the following list: ["ERROR", "UNKNOWN"]?"

